In [53]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI API key here and hit enter:")

In [60]:
!pip install openai
!pip install chromadb


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
  error: subprocess-exited-with-error
  
  × Building wheel for hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for hnswlib
ERROR: Could not build wheels for hnswlib, which is required to install pyproject.toml-based projects

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached chromadb-0.3.26-py3-none-any.whl (123 kB)
  Using cached pandas-2.0.2-cp311-cp311-win_amd64.whl (10.6 MB)
  Using cached hnswlib-0.7.0.tar.gz (33 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached clickhouse_connect-0.6.0-cp311-cp311-win_amd64.whl (223 kB)
  Using cached duckdb-0.8.0-cp311-cp311-win_amd64.whl (9.8 MB)
  Using cached fastapi-0.96.0-py3-none-any.whl (57 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached posthog-3.0.1-py2.py3-none-any.whl (37 kB)
  Using cached pulsar_client-3.2.0-cp311-cp311-win_amd64.whl (3.4 MB)
  Using cached onnxruntime-1.15.0-cp311-cp311-win_amd64.whl (6.7 MB)
  Using cached tokenizers-0.13.3-cp311-cp311-win_amd64.whl (

In [2]:
import os
import pathlib
import re

from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [3]:
!pip install pypdf
!pip install tiktoken


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# PyPdfloader

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("cucm_b_troubleshooting-guide-1251-extracted (1).pdf")
pages = loader.load_and_split()

## Davinvi v1

In [8]:
import tiktoken
# create a GPT-4 encoder instance
enc = tiktoken.encoding_for_model("gpt-4")

total_word_count = sum(len(doc.page_content.split()) for doc in pages)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in pages)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.2 / 1000}")


Total word count: 8501

Estimated tokens: 12273

Estimated cost of embedding: $2.4546


## Ada v2

In [9]:
total_word_count = sum(len(doc.page_content.split()) for doc in pages)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in pages)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0004  / 1000}")


Total word count: 8501

Estimated tokens: 12273

Estimated cost of embedding: $0.0049092


## Ada v1

In [10]:
total_word_count = sum(len(doc.page_content.split()) for doc in pages)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in pages)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0040  / 1000}")


Total word count: 8501

Estimated tokens: 12273

Estimated cost of embedding: $0.049092


## Babbage v1

In [11]:
total_word_count = sum(len(doc.page_content.split()) for doc in pages)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in pages)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0050  / 1000}")


Total word count: 8501

Estimated tokens: 12273

Estimated cost of embedding: $0.061365


## Curie v1

In [12]:
total_word_count = sum(len(doc.page_content.split()) for doc in pages)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in pages)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.020  / 1000}")


Total word count: 8501

Estimated tokens: 12273

Estimated cost of embedding: $0.24546


# PyMuPDF

In [13]:
!pip install PyMuPDF


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("cucm_b_troubleshooting-guide-1251-extracted (1).pdf")
data = loader.load()

In [15]:
total_word_count = sum(len(doc.page_content.split()) for doc in data)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in data)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")



Total word count: 8723

Estimated tokens: 12144


In [16]:
!pip install PyPDF2

     -------------------------------------- 232.6/232.6 kB 2.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import PyPDF2

def read_pdf(pdf_file):
  """Reads a PDF file and returns the text content.

  Args:
    pdf_file: The path to the PDF file to read.

  Returns:
    The text content of the PDF file.
  """

  with open(pdf_file, 'rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)
    text = ''
    for page in pdf_reader.pages:
      text += page.extract_text()

    return text

if __name__ == '__main__':
  pdf_file = 'cucm_b_troubleshooting-guide-1251-extracted (1).pdf'
  text = read_pdf(pdf_file)

  print(text)

CHAPTER 1
Troubleshooting Overview
This section provides the necessary background information and available resources to troubleshoot theCisco
Unified Communications Manager .
•Cisco Unified Serviceability, on page 1
•Cisco Unified Communications Operating System Administration, on page 2
•General Model of Problem Solving, on page 2
•Network Failure Preparation, on page 3
•Where to Find More Information, on page 3
Cisco Unified Serviceability
Cisco Unified Serviceability, a web-based troubleshooting tool for Unified Communications Manager, provides
the following functionality to assist administrators troubleshoot system problems:
• Saves Unified Communications Manager services alarms and events for troubleshooting and provi des
alarm message definitions.
•Saves Unified Communications Manager services trace information to various log files for troubleshooting .
Administrators can configure, collect, and view trace information.
• Monitors real-time behavior of the components in a Unified

## CharacterTextSplitter

In [47]:

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

In [48]:
texts = text_splitter.create_documents([text])
print(texts[0])

page_content='CHAPTER 1\nTroubleshooting Overview\nThis section provides the necessary background information and available resources to troubleshoot theCisco\nUnified Communications Manager .\n•Cisco Unified Serviceability, on page 1\n•Cisco Unified Communications Operating System Administration, on page 2\n•General Model of Problem Solving, on page 2\n•Network Failure Preparation, on page 3\n•Where to Find More Information, on page 3\nCisco Unified Serviceability\nCisco Unified Serviceability, a web-based troubleshooting tool for Unified Communications Manager, provides\nthe following functionality to assist administrators troubleshoot system problems:\n• Saves Unified Communications Manager services alarms and events for troubleshooting and provi des\nalarm message definitions.\n•Saves Unified Communications Manager services trace information to various log files for troubleshooting .\nAdministrators can configure, collect, and view trace information.\n• Monitors real-time behavior 

In [49]:
split_docs = text_splitter.split_documents(texts)

In [50]:
enc = tiktoken.encoding_for_model("gpt-4")

total_word_count = sum(len(doc.page_content.split()) for doc in split_docs)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in split_docs)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0004 / 1000}")


Total word count: 8473

Estimated tokens: 12273

Estimated cost of embedding: $0.0049092


## Create Vector Store using OpenAI

In [71]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [72]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(split_docs, embeddings)  

In [79]:
query = "Audit Logging"
docs = db.similarity_search(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [80]:
docs[0]

Document(page_content='CHAPTER 1\nTroubleshooting Overview\nThis section provides the necessary background information and available resources to troubleshoot theCisco\nUnified Communications Manager .\n•Cisco Unified Serviceability, on page 1\n•Cisco Unified Communications Operating System Administration, on page 2\n•General Model of Problem Solving, on page 2\n•Network Failure Preparation, on page 3\n•Where to Find More Information, on page 3\nCisco Unified Serviceability\nCisco Unified Serviceability, a web-based troubleshooting tool for Unified Communications Manager, provides\nthe following functionality to assist administrators troubleshoot system problems:\n• Saves Unified Communications Manager services alarms and events for troubleshooting and provi des\nalarm message definitions.\n•Saves Unified Communications Manager services trace information to various log files for troubleshooting .\nAdministrators can configure, collect, and view trace information.\n• Monitors real-time 

AttributeError: 'Chroma' object has no attribute 'save_local'

## RecursiveCharacterTextSplitter

In [27]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 8.7 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ------------------------------------- 298.0/298.0 kB 18.0 MB/s eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB 4.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.create_documents([text])
print(texts[0])

page_content='CHAPTER 1\nTroubleshooting Overview\nThis section provides the necessary background information and available resources to troubleshoot theCisco\nUnified Communications Manager .\n•Cisco Unified Serviceability, on page 1\n•Cisco Unified Communications Operating System Administration, on page 2\n•General Model of Problem Solving, on page 2\n•Network Failure Preparation, on page 3\n•Where to Find More Information, on page 3\nCisco Unified Serviceability\nCisco Unified Serviceability, a web-based troubleshooting tool for Unified Communications Manager, provides\nthe following functionality to assist administrators troubleshoot system problems:\n• Saves Unified Communications Manager services alarms and events for troubleshooting and provi des\nalarm message definitions.\n•Saves Unified Communications Manager services trace information to various log files for troubleshooting .\nAdministrators can configure, collect, and view trace information.' metadata={}


In [43]:
split_text = text_splitter.split_text(text)
split_text[:2]

['CHAPTER 1\nTroubleshooting Overview\nThis section provides the necessary background information and available resources to troubleshoot theCisco\nUnified Communications Manager .\n•Cisco Unified Serviceability, on page 1\n•Cisco Unified Communications Operating System Administration, on page 2\n•General Model of Problem Solving, on page 2\n•Network Failure Preparation, on page 3\n•Where to Find More Information, on page 3\nCisco Unified Serviceability\nCisco Unified Serviceability, a web-based troubleshooting tool for Unified Communications Manager, provides\nthe following functionality to assist administrators troubleshoot system problems:\n• Saves Unified Communications Manager services alarms and events for troubleshooting and provi des\nalarm message definitions.\n•Saves Unified Communications Manager services trace information to various log files for troubleshooting .\nAdministrators can configure, collect, and view trace information.',
 '• Monitors real-time behavior of the co

In [45]:
enc = tiktoken.encoding_for_model("gpt-4")

total_word_count = sum(len(doc.page_content.split()) for doc in split_docs)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in split_docs)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0004 / 1000}")


Total word count: 8473

Estimated tokens: 12273

Estimated cost of embedding: $0.0049092
